In [1]:
import gym
import retro
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from retro import data
import random
from collections import deque


class SMBAI(nn.Module):
    def __init__(self):
        super(SMBAI, self).__init__()
        self.Conv1 = nn.Conv2d(3, 9, 5, groups=3)
        self.Conv2 = nn.Conv2d(9, 45, 5)
        self.Conv3 = nn.Conv2d(45, 90, 5)
        self.MaxPool = nn.MaxPool2d(16)
        self.Lin1 = nn.Linear(16380, 5000)
        self.Lin2 = nn.Linear(5000, 2000)
        self.Lin3 = nn.Linear(2000, 500)
        self.Lin4 = nn.Linear(500, 18)
        
    def forward(self, x):
        x = self.Conv1(x)
        x = self.Conv2(x)
        x = self.Conv3(x)
        x = self.MaxPool(x)
        x = x.view(-1, 16380)
        x = self.Lin1(x)
        x = self.Lin2(x)
        x = self.Lin3(x)
        x = self.Lin4(x)
        return x
def ChooseButton(x, epsilon):
    if random.uniform(0, 1) <= epsilon:
        y = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        randb = random.randint(0, 8)
        y[randb] = 1
    else:
        x = x.squeeze(0)
        x = x.view(9, 2)
        y = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        z = 0
        for button in range(len(x)):
            y[button] = (((x[button] == float(torch.max(x[button]))).nonzero()).tolist())[0][0] 
        if y[7] == 1 and y[8] == 1: #Make sure L and R aren't both inputted
            if torch.max(x[7]) > torch.max(x[8]):
                 y[8] = 0
            else:
                y[7] = 0
    global epsilon_min
    global epsilon_decay
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay
    return y, epsilon

In [33]:
try:
    env
except NameError:
    env = retro.make(game='SuperMarioBros-Nes', state='Level1-1')
else:
    env.close()
    env = retro.make(game='SuperMarioBros-Nes', state='Level1-1')
epsilon = 1.00
epsilon_min = .001
epsilon_decay = .999
gamma = .01
Qlog = deque(maxlen=1000)
AI = SMBAI()
AI = AI.cuda()
LossF1 = nn.MSELoss()
OptimF1 = optim.Adam(AI.parameters(), lr=.01)
frameval = 1000
frames = frameval
epochs = 100
batch_size = 100
for x in range(epochs):
    done = False
    pvals = env.reset() # 224x240x3
    pvals = torch.from_numpy(pvals)
    pvals = pvals.view(3, 224, 240)
    pvals = torch.tensor(pvals, dtype = torch.float32, device = 'cuda')
    output = AI(pvals.unsqueeze(0))
    button, epsilon = ChooseButton(output, epsilon)
    frames = frameval
    while not done:
        frames = frames - 1
        obs, rew, done, info = env.step(button)
        Qlog.append((pvals, button, rew, obs, done))
        reward = rew if not done else -100
        #pvals = torch.from_numpy(obs)
        #pvals = pvals.view(3, 224, 240)
        #pvals = torch.LongTensor(pvals, device = 'cuda')
        output = AI(pvals.unsqueeze(0))
        button, epsilon = ChooseButton(output, epsilon)
        if len(Qlog) > batch_size:
            Qbatch = random.sample(Qlog, batch_size)
            for qstate, qactions, qreward, qnextstate, qdone in Qbatch:
                #qstate = torch.from_numpy(qstate)
                qstate = torch.reshape(qstate, (3, 224, 240))
                qstate = qstate.cuda()
                qnextstate = torch.from_numpy(qnextstate)
                qnextstate = torch.reshape(qnextstate, (3, 224, 240))
                target = qreward
                if not done:
                    #target = (qreward + gamma * torch.max(AI(qnextstate.unsqueeze(0)).data[0]))
                    # Find the action with the best reward given the next state
                # Set target to be action with best reward
                self.opt.zero_grad()
                out = AI(Variable(torch.Tensor(qstate.unsqueeze(0))))
                loss = LossF1(out, Variable(torch.Tensor(target_actual)))
                LossF1.backward()
                OptimF1.step()
        #env.render()
        if frames == 0:
            done = True
        if done:
            print("episode: {}/{}, score: {}, e: {}".format(x, epochs, reward, epsilon))

torch.save(AI.state_dict(), "./SMBAI.backup1.pth")
env.close()

NameError: name 'action' is not defined